The purpose of this project is to analyze the performance of various schools in New York city by analyzing students' SAT score(A test for university's admissions process). This test is important for a variety of stakeholders, including policy and education professionals, researchers, government, and even parents considering which school their children should attend. 

The dataset contain columns such as school name, borough, building code, average math, reading, writing score and percentage of students completing the SAT. 

In [10]:
# !pip install psycopg2
import psycopg2
import pandas
import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Setting up connection to Postgresql database
conn = psycopg2.connect(database = 'nyc_schools', 
                        user = 'postgres', 
                        host= 'localhost',
                        password = 'Redbuble27',
                        port = 5432)

In [25]:
# Select all columns from the database
# Display only the first ten rows
cur = conn.cursor()
querry = ('SELECT * FROM schools LIMIT 10;')
conn.commit()

d = pandas.read_sql_query(querry,conn)
display(type(d))
display(pandas.DataFrame.head(d))

pandas.core.frame.DataFrame

,school_name,borough,building_code,average_math,average_reading,average_writing,percent_tested
0,"New Explorations into Science, Technology and ...",Manhattan,M022,657,601,601,NaN
1,Essex Street Academy,Manhattan,M445,395,411,387,78.9
2,Lower Manhattan Arts Academy,Manhattan,M445,418,428,415,65.1
3,High School for Dual Language and Asian Studies,Manhattan,M445,613,453,463,95.9
4,Henry Street School for International Studies,Manhattan,M056,410,406,381,59.7


##  Missing values
It looks like the first school in the database had no data in the percent_tested column!

The reason behind it may indicating schools that did not report the percentage of students tested.

To understand whether this missing data problem is widespread in New York, I also calculate the total number of schools in the database.

In [30]:
# Count rows with percent_tested missing and total number of schools
querry = (''' 
SELECT 
    COUNT(*) - COUNT(percent_tested) AS num_tested_missing,
    COUNT(*) AS num_schools
FROM schools;
''')
conn.commit()

d = pandas.read_sql_query(querry,conn)
display(type(d))
display(pandas.DataFrame.head(d))

pandas.core.frame.DataFrame

,num_tested_missing,num_schools
0,20,375


## Schools by building code
There are 20 schools with missing data for percent_tested, which only makes up 5% of all rows in the database.

The first ten rows of the database, several had the same value in the building_code column, suggesting there are multiple schools based in the same location. Let's find out how many unique school locations exist.

Out of 375 schools, only 233 (62%) have a unique building_code!

In [31]:
querry = (''' 
SELECT
    COUNT(DISTINCT building_code) AS num_school_buildings
FROM schools;
''')
conn.commit()

d = pandas.read_sql_query(querry,conn)
display(type(d))
display(pandas.DataFrame.head(d))

pandas.core.frame.DataFrame

,num_school_buildings
0,233


## Best schools for math

Rank school by their performance based on their students' average math score

Schools with an average math score of at least 80% (out of 800 schools).

In [32]:
querry = (''' 
SELECT 
    school_name,
    average_math
FROM schools
WHERE average_math >= 640
ORDER BY average_math DESC;
''')
conn.commit()

d = pandas.read_sql_query(querry,conn)
display(type(d))
display(pandas.DataFrame.head(d))

pandas.core.frame.DataFrame

,school_name,average_math
0,Stuyvesant High School,754
1,Bronx High School of Science,714
2,Staten Island Technical High School,711
3,Queens High School for the Sciences at York Co...,701
4,"High School for Mathematics, Science, and Engi...",683


## Lowest reading score
Only ten public schools in New York City with an average math score of at least 640!

In [33]:
# Find lowest average_reading
querry = (''' 
SELECT MIN(average_reading) AS lowest_reading
FROM schools;
''')
conn.commit()

d = pandas.read_sql_query(querry,conn)
display(type(d))
display(pandas.DataFrame.head(d))

pandas.core.frame.DataFrame

,lowest_reading
0,302


## School with highest writting score
The lowest average score for reading across schools in New York City is less than 40% of the total available points!

In [34]:
querry = (''' 
SELECT 
    school_name,
    MAX(average_writing) AS max_writing
FROM schools
GROUP BY school_name
ORDER BY max_writing DESC
LIMIT 1;
''')
conn.commit()

d = pandas.read_sql_query(querry,conn)
display(type(d))
display(pandas.DataFrame.head(d))

pandas.core.frame.DataFrame

,school_name,max_writing
0,Stuyvesant High School,693


## Top 10 schools

The same school that got the top writing score also have highest math score, Stuyvesant High School. Stuyvesant is widely known as a perennial top school in New York.

Other schools that also have the top performance as Stuyvesant. Scores across reading, writing, and math.

In [35]:
querry = (''' 
SELECT
    school_name,
    (average_math + average_reading + average_writing) AS average_sat
FROM schools
GROUP BY school_name
ORDER BY average_sat DESC
LIMIT 10;
''')
conn.commit()

d = pandas.read_sql_query(querry,conn)
display(type(d))
display(pandas.DataFrame.head(d))

pandas.core.frame.DataFrame

,school_name,average_sat
0,Stuyvesant High School,2144
1,Staten Island Technical High School,2041
2,Bronx High School of Science,2041
3,High School of American Studies at Lehman College,2013
4,Townsend Harris High School,1981


## Ranking boroughs
There are four schools with average SAT scores of over 2000! Below is the performance by New York City borough.

Number of schools and the average SAT score per borough!

In [36]:
querry = (''' 
SELECT
    borough,
    COUNT(*) AS num_schools,
    (SUM(average_math) + SUM(average_reading) + SUM(average_writing)) / COUNT(*) AS average_borough_sat
FROM schools
GROUP BY borough
ORDER BY average_borough_sat DESC;
''')
conn.commit()

d = pandas.read_sql_query(querry,conn)
display(type(d))
display(pandas.DataFrame.head(d))

pandas.core.frame.DataFrame

,borough,num_schools,average_borough_sat
0,Staten Island,10,1439
1,Queens,69,1345
2,Manhattan,89,1340
3,Brooklyn,109,1230
4,Bronx,98,1202


It appears that schools in Staten Island, on average, produce higher scores across all three categories. However, there are only 10 schools in Staten Island, compared to an average of 91 schools in the other four boroughs and 109 schools in Brooklyn.

To find out the students' performance in Brooklyn, this query will show the top five schools based on average math score

In [38]:
querry = (''' 
SELECT
    school_name,
    average_math
FROM schools
WHERE borough = 'Brooklyn'
GROUP BY school_name
ORDER BY average_math DESC
LIMIT 5;
''')
conn.commit()

d = pandas.read_sql_query(querry,conn)
display(type(d))
display(pandas.DataFrame.head(d))

pandas.core.frame.DataFrame

,school_name,average_math
0,Brooklyn Technical High School,682
1,Brooklyn Latin School,625
2,Leon M. Goldstein High School for the Sciences,563
3,Millennium Brooklyn High School,553
4,Midwood High School,550
